In [ ]:
import torch
import numpy as np
from chronos import Chronos2Pipeline
from fusiontimeseries.lib.config import FTSConfig

torch.cuda.empty_cache()
fts_config = FTSConfig()
fts_config.prediction_length = 64

In [ ]:
model_slug = "amazon/chronos-2"

In [ ]:
pipeline: Chronos2Pipeline = Chronos2Pipeline.from_pretrained(
    pretrained_model_name_or_path=model_slug,
    device_map=fts_config.device,
    dtype=torch.bfloat16,
)

In [ ]:
from fusiontimeseries.lib.dataset import TimeseriesDataset

benchmark_flux_traces = TimeseriesDataset.get_benchmark_flux_traces(config=fts_config)

In [ ]:
from typing import Literal


START_TIMESTAMP: int = 80
forecasts: dict[Literal["id", "ood"], dict[int, np.ndarray]] = {
    "id": {},
    "ood": {},
}

for distribution, samples in benchmark_flux_traces.items():
    for sample_id, flux_trace in samples.items():
        time_series: np.ndarray = np.array(flux_trace.energy_flux)

        ctx = torch.tensor(time_series[:START_TIMESTAMP], dtype=torch.float32)

        while len(ctx) < len(time_series):
            # quantile_outputs: list(n_variates, n_quantiles, prediction_length)
            quantile_outputs: list[torch.Tensor] = pipeline.predict(
                # inputs: (n_series, n_variates, context_length)
                inputs=ctx.unsqueeze(0).unsqueeze(0),
                prediction_length=fts_config.prediction_length,
            )
            quantiles: torch.Tensor = quantile_outputs[0]

            median_quantile_idx: int = quantiles.shape[1] // 2
            median_forecast: torch.Tensor = quantiles[0, median_quantile_idx, :].cpu()

            ctx = torch.cat((ctx, median_forecast), dim=0)

        forecast: np.ndarray = ctx[: len(time_series)].numpy()
        forecasts[distribution][sample_id] = forecast

In [ ]:
from datetime import datetime
from fusiontimeseries.benchmarking.benchmark_utils import rmse_with_standard_error

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results: dict[str, dict[Literal["rsme", "rsme_se"], float]] | str = {
    "id": {},
    "ood": {},
    "timestamp": timestamp,
    "prediction_length": fts_config.prediction_length,
}  # type: ignore

for distribution, samples in benchmark_flux_traces.items():
    y_true: list[float] = []
    y_pred: list[float] = []

    for sample_id, flux_trace in samples.items():
        time_series: np.ndarray = np.array(flux_trace.energy_flux)
        target_mean = time_series[-fts_config.pred_tail_timestamps :].mean()
        forecast_mean = forecasts[distribution][sample_id][
            -fts_config.pred_tail_timestamps :
        ].mean()
        y_true.append(target_mean)
        y_pred.append(forecast_mean)

    rsme, rsme_se = rmse_with_standard_error(np.array(y_true), np.array(y_pred))
    results[distribution]["rsme"] = rsme  # type: ignore
    results[distribution]["rsme_se"] = rsme_se  # type: ignore
results

In [ ]:
import json
from pathlib import Path

# Prepare results
model_name_clean = model_slug.replace("/", "_")

# Save results to JSON
data_dir = Path(".").resolve() / "results" / model_name_clean
data_dir.mkdir(parents=True, exist_ok=True)
results_file = data_dir / f"{timestamp}_{model_name_clean}_benchmark_results.json"
with open(results_file, "w") as f:
    json.dump(results, f, indent=2)

print(f"Results saved to: {results_file}")

In [ ]:
# create and save plots of forecasts vs true values
import matplotlib.pyplot as plt

plot_dir = data_dir / "plots"
plot_dir.mkdir(parents=True, exist_ok=True)
for distribution, samples in benchmark_flux_traces.items():
    for sample_id, flux_trace in samples.items():
        time_series: np.ndarray = np.array(flux_trace.energy_flux)
        forecast: np.ndarray = forecasts[distribution][sample_id]

        plt.figure(figsize=(10, 5))
        plt.plot(time_series, label="True Values")
        plt.plot(forecast, label="Forecast", linestyle="--")
        plt.title(
            f"Forecast vs True Values ({distribution.upper()}) - Sample {sample_id}"
        )
        plt.xlabel("Time")
        plt.ylabel("Energy Flux")
        plt.legend()
        plt.grid()

        plot_file = (
            plot_dir
            / f"{timestamp}_{model_name_clean}_{distribution}_sample_{sample_id}_forecast.png"
        )
        plt.savefig(plot_file)
        plt.close()